# Introduction

You are an urban safety planner in Japan, and you are analyzing which areas of Japan need extra earthquake reinforcement.  Which areas are both high in population density and prone to earthquakes?

<center>
<img src="https://i.imgur.com/Kuh9gPj.png" width="450"><br/>
</center>

Before you get started, run the code cell below to set everything up.

In [ ]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth, Circle
from folium.plugins import HeatMap

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex3 import *

In [ ]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')



### 1) Do earthquakes coincide with plate boundaries?

Run the code cell below to create a DataFrame `plate_boundaries` that shows global plate boundaries.  The "coordinates" column is a list of (latitude, longitude) locations along the boundaries.

In [ ]:
plate_boundaries = gpd.read_file("../input/geospatial-learn-course-data/Plate_Boundaries/Plate_Boundaries/Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head(10)

Next, run the code cell below without changes to load the historical earthquake data into a DataFrame `earthquakes`.

In [ ]:
earthquakes = pd.read_csv("../input/geospatial-learn-course-data/earthquakes1970-2014.csv", parse_dates=["DateTime"])
earthquakes.head()

The code cell below visualizes the plate boundaries on a map.  Use all of the earthquake data to add a heatmap to the same map, to determine whether earthquakes coincide with plate boundaries.  

In [ ]:
# Create a base map with plate boundaries
m_1 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_1)

# Add a heatmap to the map
HeatMap(data=earthquakes[['Latitude', 'Longitude']], radius=15).add_to(m_1)

# Show the map
embed_map(m_1, 'q_1.html')

### 2) Is there a relationship between earthquake depth and proximity to a plate boundary in Japan?





In [ ]:
# Create a base map with plate boundaries
m_2 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_2)
    
#Add a map to visualize earthquake depth
def color_producer(val):
    if val < 50:
        return 'forestgreen'
    elif val < 100:
        return 'darkorange'
    else:
        return 'darkred'

for i in range(0,len(earthquakes)):
    Circle(
        location=[earthquakes.iloc[i]['Latitude'],earthquakes.iloc[i]['Longitude']],
        radius=20,
        color=color_producer(earthquakes.iloc[i]['Depth'])).add_to(m_2)

# View the map
embed_map(m_2, 'q_2.html')

### 3) Which prefectures have high population density?


In [ ]:
# GeoDataFrame with prefecture boundaries
prefectures = gpd.read_file("../input/geospatial-learn-course-data/japan-prefecture-boundaries/japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

The next code cell creates a DataFrame `stats` containing the population, area (in square kilometers), and population density (per square kilometer) for each Japanese prefecture.  Run the code cell without changes.

In [ ]:
# DataFrame containing population of each prefecture
population = pd.read_csv("../input/geospatial-learn-course-data/japan-prefecture-population.csv")
population.set_index('prefecture', inplace=True)

# Calculate area (in square kilometers) of each prefecture
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

# Add density (per square kilometer) of each prefecture
stats['density'] = stats["population"] / stats["area_sqkm"]
stats.head()

Use the next code cell to create a choropleth map to visualize population density.

In [ ]:
# Create a base map
m_3 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

#Create a choropleth map to visualize population density
Choropleth(geo_data=prefectures.__geo_interface__, 
           data=stats['density'], 
           key_on="feature.id",
           fill_color='YlGnBu', 
           legend_name='Population Density (per square kilometer)'
          ).add_to(m_3)

# View the map
embed_map(m_3, 'q_3.html')

Tokyo,Kanagawa and Osaka have relatively higher density than the others.

### 4) Which high-density prefecture is prone to high-magnitude earthquakes?

Create a map to suggest one prefecture that might benefit from earthquake reinforcement.  We map should visualize both density and earthquake magnitude.

In [ ]:
# Create a base map
m_4 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

#Create a map
def color_producer(val):
    if val > 6.5:
        return 'red'
    else:
        return 'green'

for i in range(0,len(earthquakes)):
    Circle(
        location=[earthquakes.iloc[i]['Latitude'],earthquakes.iloc[i]['Longitude']],
        radius=20,
        color=color_producer(earthquakes.iloc[i]['Magnitude'])).add_to(m_4)
    
Choropleth(geo_data=prefectures.__geo_interface__, 
           data=stats['density'], 
           key_on="feature.id",
           fill_color='BuPu', 
           legend_name='Population Density (per square kilometer)'
          ).add_to(m_4)


# View the map
embed_map(m_4, 'q_4.html')

Which prefecture do you recommend for extra earthquake reinforcement?

Osaka near to high magnitude earthquake while Kanagawa may be under tsunami risk. Additionally, Tokyo has a high population density. For three prefectures may take extra earthquake reinforcement.